In [1]:
!pip install transformers torch sentence-transformers faiss-cpu wikipedia datetime pytz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.2 MB/s eta 0:00:00
   

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipedia
from datetime import datetime
import pytz
import random
import re
import warnings
warnings.filterwarnings('ignore')

class Stellaris:
    def __init__(self):
        # Model Initialization (Optimized for speed)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = "microsoft/DialoGPT-large"  # More powerful than medium
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name).to(self.device)
        self.generator = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == "cuda" else -1,
        )

        # Knowledge Retrieval (RAG - Retrieval Augmented Generation)
        self.retriever = SentenceTransformer('all-MiniLM-L6-v2').to(self.device)
        self.knowledge_index = faiss.IndexFlatL2(384)  # Vector dimension for MiniLM

        # Conversation Memory
        self.chat_history = []
        self.max_history = 6  # Last 3 exchanges

        # Bot Personality & Config
        self.bot_name = "Stellaris"
        self.user_name = "Traveler"  # Cosmic-themed user name
        self.timezone = pytz.timezone('UTC')  # Can be changed

        # Predefined Responses & Skills
        self.skills = {
            "time": self.get_current_time,
            "date": self.get_current_date,
            "weather": self.get_weather_response,
            "joke": self.tell_joke,
            "calculate": self.simple_math,
            "wiki": self.get_wikipedia_summary,
            "fact": self.tell_space_fact,
        }

        # Welcome Message
        self.welcome_msg = f"""
        🌠 **Welcome to {self.bot_name} - Your Guide Through the Cosmos of Knowledge!** 🌠
        - Ask me anything (science, jokes, time, math, etc.)
        - Type `/help` for commands
        - Type `quit` to exit
        Current Stardate: {self.get_current_date()} | Cosmic Time: {self.get_current_time()}
        """

    # ==== Knowledge & Utility Functions ====
    def get_current_time(self):
        return datetime.now(self.timezone).strftime("%H:%M:%S")

    def get_current_date(self):
        return datetime.now(self.timezone).strftime("%Y-%m-%d")

    def get_weather_response(self):
        responses = [
            "I don’t have real-time weather, but on Mars, it’s probably -80°F!",
            "In the Andromeda galaxy, weather forecasts are always spectacular!",
            "Earth’s weather is unpredictable, but I hope it’s pleasant where you are!"
        ]
        return random.choice(responses)

    def tell_joke(self):
        jokes = [
            "Why don’t aliens eat clowns? Because they taste funny!",
            "How do you throw a space party? You planet!",
            "Why did the sun go to school? To get a little brighter!"
        ]
        return random.choice(jokes)

    def tell_space_fact(self):
        facts = [
            "A day on Venus is longer than its year!",
            "Neutron stars are so dense, a teaspoon would weigh a billion tons.",
            "There are more stars in the universe than grains of sand on all Earth’s beaches."
        ]
        return f"✨ Cosmic Fact: {random.choice(facts)}"

    def simple_math(self, query):
        try:
            expr = re.sub(r'[^0-9+\-*/(). ]', '', query)
            return f"⚡ Calculated: {eval(expr)}"  # Caution: Basic eval usage
        except:
            return "🚫 Error: I couldn’t compute that. Try something like '5 + 3'."

    def get_wikipedia_summary(self, query):
        try:
            summary = wikipedia.summary(query, sentences=2)
            return f"📚 Galactic Archives: {summary}"
        except:
            return "🚫 Error: No cosmic data found. Try another query."

    # ==== Response Generation Logic ====
    def retrieve_knowledge(self, query):
        """Retrieve relevant facts if available"""
        if "time" in query.lower():
            return f"⏳ Cosmic Time: {self.get_current_time()}"
        elif "date" in query.lower():
            return f"📅 Stardate: {self.get_current_date()}"
        elif "weather" in query.lower():
            return self.get_weather_response()
        elif "joke" in query.lower():
            return self.tell_joke()
        elif "fact" in query.lower():
            return self.tell_space_fact()
        elif any(op in query for op in ['+', '-', '*', '/']):
            return self.simple_math(query)
        elif "who is" in query.lower() or "what is" in query.lower():
            return self.get_wikipedia_summary(query.split("is")[-1].strip())
        return None

    def generate_response(self, user_input):
        """Generate AI response with context"""
        # Check for commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            return "🌌 Farewell, Traveler! May the stars guide you back soon."

        # Check for help command
        if user_input.lower() == '/help':
            return (
                "🛸 **Stellaris Command Matrix:**\n"
                "- Ask me anything (science, history, jokes, etc.)\n"
                "- `time` → Current cosmic time\n"
                "- `date` → Today’s stardate\n"
                "- `weather` → Interstellar weather\n"
                "- `joke` → Space-themed humor\n"
                "- `fact` → Amazing cosmic facts\n"
                "- `calculate <expression>` → Math solver\n"
                "- `who is <person>` → Galactic archives lookup\n"
            )

        # Retrieve knowledge if applicable
        knowledge_response = self.retrieve_knowledge(user_input)
        if knowledge_response:
            return knowledge_response

        # Generate AI response with conversation history
        prompt = (
            f"{' '.join(self.chat_history[-self.max_history:])}\n"
            f"Traveler: {user_input}\n"
            f"Stellaris:"
        )

        response = self.generator(
            prompt,
            max_length=150,
            num_return_sequences=1,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
        )

        # Extract and clean response
        ai_response = response[0]['generated_text'].split("Stellaris:")[-1].strip()
        ai_response = re.sub(r'<\|.*?\|>', '', ai_response)  # Remove special tokens

        # Ensure response isn't just repeating input
        if ai_response.lower() == user_input.lower():
            return "🪐 Fascinating! Tell me more about your cosmic query."

        return ai_response

    # ==== Main Chat Loop ====
    def start_chat(self):
        print(self.welcome_msg)

        while True:
            try:
                user_input = input(f"{self.user_name}: ").strip()
                if not user_input:
                    continue

                # Get response
                response = self.generate_response(user_input)
                print(f"{self.bot_name}: {response}")

                # Update chat history
                self.chat_history.extend([user_input, response])
                if len(self.chat_history) > self.max_history * 2:
                    self.chat_history = self.chat_history[-self.max_history * 2:]

                # Exit condition
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    break

            except KeyboardInterrupt:
                print(f"\n{self.bot_name}: 🌠 Until our paths cross again among the stars!")
                break
            except Exception as e:
                print(f"{self.bot_name}: ⚠️ Cosmic interference detected! Let's try again.")
                continue

In [ ]:
# Initialize & run
cosmic_bot = Stellaris()
cosmic_bot.start_chat()